インポート

In [8]:
from urllib.request import urlopen, Request
import pandas as pd
from bs4 import BeautifulSoup
import re
import time
from tqdm.notebook import tqdm
from pathlib import Path
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchWindowException
from selenium.webdriver.common.by import By
import traceback
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
url = "https://db.netkeiba.com/race/202406010111/"
headers={"User-Agent":"Mozilla/5.0"}
request=Request(url, headers=headers)
html = urlopen(request).read()
pd.read_html(html)[2]

,0,1,2,3
0,ワイド,3 - 7 3 - 15 7 - 15,"1,450 1,310 1,870",22 18 27
1,馬単,3 → 7,8030,38
2,三連複,3 - 7 - 15,12230,52
3,三連単,3 → 7 → 15,65470,252


## 開催日一覧を取得

In [20]:
url="https://race.netkeiba.com/top/calendar.html?pid=race_calendar&year=2023&month=1"
headers={"User-Agent":"Mozilla/5.0"}
request=Request(url, headers=headers)
html = urlopen(request).read()

soup=BeautifulSoup(html)
soup

HTTPError: HTTP Error 400: Bad Request

In [3]:
def scrape_kaisai_date(from_, to_):
    kaisai_date_list = []
    for date in tqdm(pd.date_range(from_, to_, freq="MS")):
        year = date.year
        month = date.month
        url = f'https://race.netkeiba.com/top/calendar.html?year={year}&month={month}'
        headers={"User-Agent":"Mozilla/5.0"}
        request=Request(url, headers=headers)
        html = urlopen(request).read() #スクレイピング
        time.sleep(1)
        soup = BeautifulSoup(html)
        a_list = soup.find("table", class_="Calendar_Table").find_all("a")
        for a in a_list:
            kaisai_date = re.findall(r"kaisai_date=(\d{8})", a["href"])[0]
            kaisai_date_list.append(kaisai_date)

    return kaisai_date_list

In [13]:
scrape_kaisai_date(from_="2023-1", to_="2024-12")

  0%|          | 0/24 [00:00<?, ?it/s]

['20230105',
 '20230107',
 '20230108',
 '20230109',
 '20230114',
 '20230115',
 '20230121',
 '20230122',
 '20230128',
 '20230129',
 '20230204',
 '20230205',
 '20230211',
 '20230212',
 '20230218',
 '20230219',
 '20230225',
 '20230226',
 '20230304',
 '20230305',
 '20230311',
 '20230312',
 '20230318',
 '20230319',
 '20230325',
 '20230326',
 '20230401',
 '20230402',
 '20230408',
 '20230409',
 '20230415',
 '20230416',
 '20230422',
 '20230423',
 '20230429',
 '20230430',
 '20230506',
 '20230507',
 '20230513',
 '20230514',
 '20230520',
 '20230521',
 '20230527',
 '20230528',
 '20230603',
 '20230604',
 '20230610',
 '20230611',
 '20230617',
 '20230618',
 '20230624',
 '20230625',
 '20230701',
 '20230702',
 '20230708',
 '20230709',
 '20230715',
 '20230716',
 '20230722',
 '20230723',
 '20230729',
 '20230730',
 '20230805',
 '20230806',
 '20230812',
 '20230813',
 '20230819',
 '20230820',
 '20230826',
 '20230827',
 '20230902',
 '20230903',
 '20230909',
 '20230910',
 '20230916',
 '20230917',
 '20230918',

開催ページからrace_id取得

In [14]:
url="https://race.netkeiba.com/top/race_list.html?kaisai_date=20230105"
headers={"User-Agent":"Mozilla/5.0"}
request=Request(url, headers=headers)
html = urlopen(request).read()
soup=BeautifulSoup(html)

soup.find("div",class_="RaceList_Box")

取得できないので、ChromeDriverを使う

In [15]:
pip install selenium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install webdriver-manager

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install --upgrade webdriver-manager

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [18]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchWindowException
from selenium.webdriver.common.by import By
import traceback

In [19]:
options = Options()
options.add_argument("--user-agent=Mozilla/5.0")

# 正しいバイナリファイルのパスを指定
chromedriver_path = '/home/jawa/.wdm/drivers/chromedriver/linux64/131.0.6778.264/chromedriver-linux64/chromedriver'
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=options)


# ブラウザウィンドウが存在するか確認し、再試行ロジックを追加
try:
    driver.get(url)
except NoSuchWindowException:
    print("No such window: target window already closed. Retrying...")
    # 再試行のために新しいウィンドウを開く
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)

li_list=driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")
li_list

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=131.0.6778.264)
Stacktrace:
#0 0x5601d7b611fa <unknown>
#1 0x5601d7671810 <unknown>
#2 0x5601d7658555 <unknown>
#3 0x5601d7658433 <unknown>
#4 0x5601d7673b32 <unknown>
#5 0x5601d77029e6 <unknown>
#6 0x5601d76e4323 <unknown>
#7 0x5601d76b2de0 <unknown>
#8 0x5601d76b3dbe <unknown>
#9 0x5601d7b2d12b <unknown>
#10 0x5601d7b310c7 <unknown>
#11 0x5601d7b1a6cc <unknown>
#12 0x5601d7b31c47 <unknown>
#13 0x5601d7aff67f <unknown>
#14 0x5601d7b50288 <unknown>
#15 0x5601d7b50450 <unknown>
#16 0x5601d7b60076 <unknown>
#17 0x76d44cc94ac3 <unknown>


In [52]:
li=li_list[0]
li

<selenium.webdriver.remote.webelement.WebElement (session="4e58fe73a4e383a514085bf5eb63bb8d", element="f.C08E85BF747D2411C112B646803482F8.d.D9D396522F04686AE86C25F84EC4ECC6.e.47")>

In [61]:
race_id_list=[]
for li in li_list:
    href=li.find_element(By.TAG_NAME, "a").get_attribute("href")
    race_id=re.findall(r"race_id=(\d{12})", href)[0]
    #print(race_id)
    race_id_list.append(race_id)

driver.quit()

In [ ]:
###以下からやればいい

In [5]:
import scraping

In [6]:
kaisai_date_list=scraping.scrape_kaisai_date(from_="2023-01",to_="2023-12")

  0%|          | 0/12 [00:00<?, ?it/s]

In [7]:
def scrape_race_id_list(kaisai_date_list:list[str]):
    options = Options()
    options.add_argument("--user-agent=Mozilla/5.0")
    options.add_argument("--headless")

    # 正しいバイナリファイルのパスを指定
    chromedriver_path = '/home/jawa/.wdm/drivers/chromedriver/linux64/131.0.6778.264/chromedriver-linux64/chromedriver'
    service = Service(chromedriver_path)
    race_id_list=[]

    with webdriver.Chrome(service=service, options=options) as driver:
        for kaisai_date in tqdm(kaisai_date_list):
            url=f"https://race.netkeiba.com/top/race_list.html?kaisai_date={kaisai_date}"
            try:
                driver.get(url)
                time.sleep(1)
                li_list=driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")
                for li in li_list:
                    href=li.find_element(By.TAG_NAME, "a").get_attribute("href")
                    race_id=re.findall(r"race_id=(\d{12})", href)[0]
                    #print(race_id)
                    race_id_list.append(race_id)
            except:
                print(f"stopped at {url}")
                print(traceback.format_exc())
                break
    return race_id_list

In [8]:
race_id_list=scrape_race_id_list(kaisai_date_list)

  0%|          | 0/107 [00:00<?, ?it/s]

In [9]:
import pickle
with open("race_id_list.pickle", "wb") as f:
    pickle.dump(race_id_list, f)

In [18]:
url = "https://db.netkeiba.com/race/202406010111/"
headers={"User-Agent":"Mozilla/5.0"}
request=Request(url, headers=headers)
html = urlopen(request).read()

pd.read_html(html)[0]

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
0,1,2,3,リカンカブール,牡5,56.0,津村明秀,1:58.9,NaN,7.8,5,494(-12),[西] 田中克典
1,2,4,7,ククナ,牝6,54.0,戸崎圭太,1:59.0,1/2,15.4,8,468(+6),[東] 栗田徹
2,3,8,15,マイネルクリソーラ,牡5,55.0,Ｍ．デム,1:59.1,クビ,11.1,6,458(+4),[東] 中野栄治
3,4,4,8,ボーンディスウェイ,牡5,55.0,木幡巧也,1:59.2,3/4,4.8,2,490(0),[東] 牧光二
4,5,6,12,ホウオウアマゾン,牡6,58.0,横山武史,1:59.3,1/2,113.8,14,512(-4),[西] 矢作芳人
5,6,3,6,クリノプレミアム,牝7,55.5,松岡正海,1:59.3,クビ,36.0,10,488(+2),[東] 伊藤伸一
6,7,5,10,マテンロウレオ,牡5,58.5,横山典弘,1:59.3,ハナ,7.0,3,482(-2),[西] 昆貢
7,8,7,13,カテドラル,牡8,58.5,荻野極,1:59.4,3/4,184.0,17,488(-4),[西] 池添学
8,9,6,11,ゴールデンハインド,牝4,54.0,菅原明良,1:59.5,アタマ,7.3,4,498(+6),[東] 武市康男
9,10,3,5,マイネルファンロン,牡9,56.0,丸山元気,1:59.5,アタマ,134.6,15,490(+10),[東] 手塚貴久


In [10]:
with open("race_id_list.pickle","rb") as f:
    race_id_list=pickle.load(f)

In [11]:
with open("202406010111.bin","wb") as f:
    f.write(html)

In [13]:
HTML_DIR=Path("..","data","html")

In [14]:
with open("202406010111.bin","wb") as f:
    f.write(html)

In [15]:
for race_id in race_id_list[:10]:
    url=f"https://db.netkeiba.com/race/{race_id}"
    headers={"User-Agent":"Mozilla/5.0"}
    request=Request(url, headers=headers)
    html = urlopen(request).read()
    time.sleep(1)
    with open(HTML_DIR/f"{race_id}.bin","wb") as f:
        f.write(html)

In [30]:
from pathlib import Path

def scrape_html_race(race_id_list: list[str], save_dir: Path) -> list[Path]:
    html_path_list=[]
    for race_id in tqdm(race_id_list):
        filepath=save_dir/f"{race_id}.bin"
        # binファイルがすでに存在する場合はスキップする
        if filepath.is_file():
            print("skipped:{race_id}")
        else:
            url=f"https://db.netkeiba.com/race/{race_id}"
            headers={"User-Agent":"Mozilla/5.0"}
            request=Request(url, headers=headers)
            html = urlopen(request).read()
            time.sleep(1)
            with open(filepath,"wb") as f:
                f.write(html)
            html_path_list.append(filepath)
    return html_path_list

In [31]:
html_path_list=scrape_html_race(race_id_list[:14],save_dir=HTML_DIR)

  0%|          | 0/14 [00:00<?, ?it/s]

skipped:{race_id}
skipped:{race_id}
skipped:{race_id}
skipped:{race_id}
skipped:{race_id}
skipped:{race_id}
skipped:{race_id}
skipped:{race_id}
skipped:{race_id}
skipped:{race_id}


In [29]:
html_path_list[0].is_file()

True

##1/8

In [44]:
HTML_RACE_DIR=HTML_DIR/"race"
html_paths_race=list(HTML_RACE_DIR.glob("*"))

In [45]:
len(html_paths_race)

424

In [51]:
dfs={}
for html_path in html_paths_race[:3]:
    with open(html_path, "rb") as f:
        race_id=html_path.stem
        html=f.read()
        df=pd.read_html(html)[0]
        df.index=[race_id]*len(df)
        dfs[race_id]=df
concat_df=pd.concat(dfs.values())
concat_df.index.name="race_id"

In [47]:
dfs

{'202307010403':     着 順  枠 番  馬 番         馬名  性齢  斤量    騎手     タイム     着差     単勝  人 気  \
 0     1    5    6  カレンアルカンタラ  牡3  56  川田将雅  1:54.4    NaN   10.5    5   
 1     2    8   12   ベンダバリラビア  牡3  53  今村聖奈  1:54.6  1.1/4    9.5    4   
 2     3    6    8       サグアロ  牡3  56  イーガン  1:54.8  1.1/2   17.7    8   
 3     4    8   11      プラーヴィ  牡3  56  福永祐一  1:54.9    1/2    2.5    1   
 4     5    6    7      キレナイカ  牡3  56  吉田隼人  1:55.3  2.1/2    3.5    2   
 5     6    7   10       ササヤキ  牡3  56   浜中俊  1:55.4    1/2   40.4   10   
 6     7    2    2   オーロイプラータ  牡3  56   幸英明  1:55.8  2.1/2    8.3    3   
 7     8    3    3  タマモブラックタイ  牡3  54  角田大河  1:56.0  1.1/2   65.9   11   
 8     9    1    1  サンデーファンデー  牡3  56    武豊  1:56.4  2.1/2   15.4    6   
 9    10    5    5  シリンガバルガリス  牡3  56  ムルザバ  1:56.8  2.1/2   17.6    7   
 10   11    4    4     ビキニボーイ  牡3  56  古川吉洋  1:57.5      4  231.4   12   
 11   12    7    9   シャーマンロック  牡3  56  岩田康誠  1:57.7  1.1/2   30.7    9   
 
          馬体重       

In [52]:
concat_df

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
race_id,,,,,,,,,,,,,
202307010403,1,5,6,カレンアルカンタラ,牡3,56,川田将雅,1:54.4,NaN,10.5,5,492(0),[西] 新谷功一
202307010403,2,8,12,ベンダバリラビア,牡3,53,今村聖奈,1:54.6,1.1/4,9.5,4,526(0),[西] 寺島良
202307010403,3,6,8,サグアロ,牡3,56,イーガン,1:54.8,1.1/2,17.7,8,530(+2),[西] 高野友和
202307010403,4,8,11,プラーヴィ,牡3,56,福永祐一,1:54.9,1/2,2.5,1,468(-6),[西] 杉山晴紀
202307010403,5,6,7,キレナイカ,牡3,56,吉田隼人,1:55.3,2.1/2,3.5,2,530(+8),[西] 須貝尚介
202307010403,6,7,10,ササヤキ,牡3,56,浜中俊,1:55.4,1/2,40.4,10,538(+2),[西] 畑端省吾
202307010403,7,2,2,オーロイプラータ,牡3,56,幸英明,1:55.8,2.1/2,8.3,3,532(-2),[西] 宮本博
202307010403,8,3,3,タマモブラックタイ,牡3,54,角田大河,1:56.0,1.1/2,65.9,11,528(+12),[西] 角田晃一
202307010403,9,1,1,サンデーファンデー,牡3,56,武豊,1:56.4,2.1/2,15.4,6,540(-2),[西] 音無秀孝


In [67]:
def create_results(html_path_list: list[Path]) -> pd.DataFrame:
    dfs={}
    for html_path in tqdm(html_path_list):
        with open(html_path, "rb") as f:
            race_id=html_path.stem
            html=f.read()
            df=pd.read_html(html)[0]
            df.index=[race_id]*len(df)
            dfs[race_id]=df
    concat_df=pd.concat(dfs.values())
    concat_df.index.name="race_id"
    return concat_df

In [68]:
results=create_results(html_path_list=html_paths_race)

  0%|          | 0/424 [00:00<?, ?it/s]

## 1/10

In [31]:
url="https://db.netkeiba.com/horse/2017102537"
headers={"User-Agent":"Mozilla/5.0"}
request=Request(url, headers=headers)
html = urlopen(request).read()
pd.read_html(html)[2]

,日付,開催,天 気,R,レース名,映 像,頭 数,枠 番,馬 番,オ ッ ズ,...,着差,ﾀｲﾑ 指数,通過,ペース,上り,馬体重,厩舎 ｺﾒﾝﾄ,備考,勝ち馬 (2着馬),賞金
0,2025/01/06,1中京2,雨,11,万葉S(OP),NaN,17,3,5,167.1,...,3.7,**,12-14-16-16,37.2-40.0,39.8,440(-4),NaN,NaN,ゴールデンスナップ,NaN
1,2024/11/30,5中山1,晴,11,ステイヤーズS(GII),NaN,14,6,9,292.7,...,0.8,**,12-12-10-11,38.7-35.3,35.4,444(0),NaN,NaN,シュヴァリエローズ,NaN
2,2024/11/03,5東京2,晴,11,アルゼンチン共和国杯(GII),NaN,16,3,5,246.0,...,1.4,**,14-15-15-15,29.6-36.1,36.1,444(0),NaN,NaN,ハヤヤッコ,NaN
3,2024/10/13,4東京3,晴,11,オクトーバーS(L),NaN,16,5,9,193.3,...,0.7,**,9-9-11,34.8-35.3,34.1,444(0),NaN,NaN,ボーンディスウェイ,NaN
4,2024/06/30,1函館8,曇,11,巴賞(OP),NaN,16,4,8,135.8,...,1.4,**,11-11-13-15,35.4-35.1,35.5,444(+6),NaN,NaN,ホウオウビスケッツ,NaN
5,2024/06/01,4京都1,晴,11,鳴尾記念(GIII),NaN,13,6,10,182.3,...,1.3,**,12-12-12-12,34.9-34.4,34.4,438(+2),NaN,NaN,ヨーホーレイク,NaN
6,2024/05/11,3京都7,晴,11,都大路S(L),NaN,11,8,10,114.8,...,0.8,**,9-10,35.4-33.6,33.6,436(+2),NaN,NaN,セオ,280.0
7,2024/03/03,1阪神4,晴,11,大阪城S(L),NaN,15,7,12,55.2,...,0.6,**,8-10,34.6-34.8,34.6,434(0),NaN,NaN,ステラヴェローチェ,NaN
8,2024/02/04,1小倉8,晴,11,小倉日経OP(OP),NaN,9,3,3,22.0,...,0.1,**,4-4-4-5,37.3-37.8,37.4,434(-2),NaN,NaN,ダンディズム,965.6
9,2024/01/06,1京都1,晴,10,万葉S(OP),NaN,13,4,5,60.3,...,2.6,**,5-5-6-9,37.1-37.0,38.9,436(-2),NaN,NaN,メイショウブレゲ,NaN


In [32]:
url="https://db.netkeiba.com/race/202306010111/"
headers={"User-Agent":"Mozilla/5.0"}
request=Request(url, headers=headers)
html = urlopen(request).read()

In [33]:
soup=BeautifulSoup(html,"lxml")
table=soup.find("table",class_="race_table_01 nk_tb_common")
table

<table cellpadding="0" cellspacing="1" class="race_table_01 nk_tb_common" summary="レース結果">
<tr class="txt_c">
<th nowrap="nowrap">着<br/>順</th>
<th nowrap="nowrap">枠<br/>番</th>
<th nowrap="nowrap">馬<br/>番</th>
<th nowrap="nowrap">馬名</th>
<th nowrap="nowrap">性齢</th>
<th nowrap="nowrap">斤量</th>
<th nowrap="nowrap">騎手</th>
<th nowrap="nowrap">タイム</th>
<th nowrap="nowrap">着差</th>
<diary_snap_cut>
<th nowrap="nowrap">ﾀｲﾑ<br/>指数<br/><img height="16" src="https://cdn.netkeiba.com/img.db//style/netkeiba.ja/image/ico_premium.gif" width="16"/></th>
<th nowrap="nowrap">通過</th>
<th nowrap="nowrap">上り</th>
</diary_snap_cut>
<th nowrap="nowrap">単勝</th>
<th nowrap="nowrap">人<br/>気</th>
<th nowrap="nowrap">馬体重</th>
<diary_snap_cut>
<th nowrap="nowrap">調教<br/>ﾀｲﾑ<br/><img height="16" src="https://cdn.netkeiba.com/img.db//style/netkeiba.ja/image/ico_premium.gif" width="16"/></th>
<th nowrap="nowrap">厩舎<br/>ｺﾒﾝﾄ<br/><img height="16" src="https://cdn.netkeiba.com/img.db//style/netkeiba.ja/image/ico_premium

In [34]:
table.find_all("a",href="/horse/2019101782/")

[<a href="/horse/2019101782/" id="umalink_202306010111" title="ラーグルフ">ラーグルフ</a>]

In [35]:
a_list=table.find_all("a",href=re.compile(r"^/horse/"))
a_list

[<a href="/horse/2019101782/" id="umalink_202306010111" title="ラーグルフ">ラーグルフ</a>,
 <a href="/horse/2017103201/" id="umalink_202306010111" title="クリノプレミアム">クリノプレミアム</a>,
 <a href="/horse/2019105366/" id="umalink_202306010111" title="フェーングロッテン">フェーングロッテン</a>,
 <a href="/horse/2017104756/" id="umalink_202306010111" title="アラタ">アラタ</a>,
 <a href="/horse/2019102879/" id="umalink_202306010111" title="マテンロウレオ">マテンロウレオ</a>,
 <a href="/horse/2019104981/" id="umalink_202306010111" title="レッドランメルト">レッドランメルト</a>,
 <a href="/horse/2018104579/" id="umalink_202306010111" title="カレンルシェルブル">カレンルシェルブル</a>,
 <a href="/horse/2017102537/" id="umalink_202306010111" title="フォワードアゲン">フォワードアゲン</a>,
 <a href="/horse/2018105060/" id="umalink_202306010111" title="ラーゴム">ラーゴム</a>,
 <a href="/horse/2017101449/" id="umalink_202306010111" title="エヴァーガーデン">エヴァーガーデン</a>,
 <a href="/horse/2016105681/" id="umalink_202306010111" title="スカーフェイス">スカーフェイス</a>,
 <a href="/horse/2017105620/" id="umalink_202306010111" title="レインカ

In [36]:
a=a_list[0]
a

<a href="/horse/2019101782/" id="umalink_202306010111" title="ラーグルフ">ラーグルフ</a>

In [37]:
a["href"]
re.findall(r"\d{10}",a["href"])[0]

'2019101782'

In [38]:
horse_id_list=[]
for a in a_list:
    horse_id=re.findall(r"\d{10}",a["href"])[0]
    #print(horse_id)
    horse_id_list.append(horse_id)
len(horse_id_list)

17

In [4]:
import create_rawdf
from pathlib import Path
%load_ext autoreload

In [5]:
HTML_RACE_DIR=Path("..", "data","html","race")

In [6]:
html_paths_race=list(HTML_RACE_DIR.glob("*.bin"))

In [7]:
results=create_rawdf.create_results(html_paths_race[:10])
results

  0%|          | 0/10 [00:00<?, ?it/s]

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師,horse_id,jockey_id,trainer_id,owner_id
race_id,,,,,,,,,,,,,,,,,
202309020109,1,7,7,キミノナハマリア,牝3,54,松山弘平,1:47.0,NaN,3.0,1.0,490(+4),[西] 千田輝彦,2020102873,01126,01120,220031
202309020109,2,3,3,ランスオブサウンド,牝3,54,団野大成,1:47.5,3,6.2,4.0,430(+2),[西] 奥村豊,2020104739,01180,01146,477006
202309020109,3,2,2,アリスヴェリテ,牝3,54,藤岡康太,1:47.5,ハナ,5.2,3.0,438(-4),[西] 中竹和也,2020105655,01116,01039,944030
202309020109,4,8,10,メテオリート,牝3,54,吉田隼人,1:47.5,アタマ,14.2,7.0,470(-12),[西] 吉岡辰弥,2020103648,01095,01176,506800
202309020109,5,8,9,サーマルソアリング,牝3,54,岩田望来,1:47.6,1/2,8.2,5.0,462(-6),[西] 藤原英昭,2020102688,01174,01055,486800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202309040909,8,5,6,スズノマーベリック,セ4,58,秋山真一,1:46.9,3/4,77.2,10.0,470(+4),[西] 石坂公一,2019102496,01019,01166,835030
202309040909,9,8,11,ゲンパチムサシ,牡5,58,藤岡佑介,1:47.0,クビ,24.1,6.0,502(-6),[西] 佐々木晶,2018102832,01093,00429,689002
202309040909,10,3,3,シゲルローズマリー,セ4,58,高倉稜,1:47.0,アタマ,51.2,8.0,466(-4),[西] 渡辺薫彦,2019101261,01130,01155,552031


In [43]:
results.columns=results.columns.str.replace(" ","")

In [9]:
save_dir=HTML_RACE_DIR=Path("..", "data","rawdf")
save_dir.mkdir()

FileExistsError: [Errno 17] File exists: '../data/rawdf'

In [10]:
results.to_csv(save_dir/"results.csv", sep="\t")

In [11]:
horse_id_list=results["horse_id"].unique()

In [12]:
len(horse_id_list)

137

In [15]:
import scraping

html_paths_horse=scraping.scrape_html_horse(horse_id_list=horse_id_list)

  0%|          | 0/137 [00:00<?, ?it/s]

In [16]:
html_paths_horse

[PosixPath('../data/html/horse/2020102873.bin'),
 PosixPath('../data/html/horse/2020104739.bin'),
 PosixPath('../data/html/horse/2020105655.bin'),
 PosixPath('../data/html/horse/2020103648.bin'),
 PosixPath('../data/html/horse/2020102688.bin'),
 PosixPath('../data/html/horse/2020103655.bin'),
 PosixPath('../data/html/horse/2020105615.bin'),
 PosixPath('../data/html/horse/2020103255.bin'),
 PosixPath('../data/html/horse/2020109003.bin'),
 PosixPath('../data/html/horse/2020102563.bin'),
 PosixPath('../data/html/horse/2020103565.bin'),
 PosixPath('../data/html/horse/2020103928.bin'),
 PosixPath('../data/html/horse/2020101557.bin'),
 PosixPath('../data/html/horse/2020103225.bin'),
 PosixPath('../data/html/horse/2020105656.bin'),
 PosixPath('../data/html/horse/2019106251.bin'),
 PosixPath('../data/html/horse/2020103052.bin'),
 PosixPath('../data/html/horse/2018102151.bin'),
 PosixPath('../data/html/horse/2019106088.bin'),
 PosixPath('../data/html/horse/2019105341.bin'),
 PosixPath('../data/

In [51]:
%autoreload

In [17]:
horse_results=create_rawdf.create_horse_results(html_path_list=html_paths_horse)

  0%|          | 0/137 [00:00<?, ?it/s]

##レース情報の取得

In [11]:
DATA_DIR=Path("..","data")

In [12]:
HTML_DIR=DATA_DIR/"html"
HTML_RACE_DIR=HTML_DIR/"race"

In [13]:
html_path_list=list(HTML_RACE_DIR.glob("*bin"))

In [14]:
html_path_list[0]

PosixPath('../data/html/race/202309020109.bin')

In [15]:
with open(html_path_list[0],"rb") as f:
    html=f.read()

In [17]:
soup=BeautifulSoup(html,"lxml")

In [21]:
soup_info=soup.find("div",class_="data_intro")
soup_info

<div class="data_intro">
<dl class="racedata fc">
<dt>
9 R
</dt>
<dd>
<h1>君子蘭賞(1勝)<!--img src="style/netkeiba.ja/image/race_grade_g2_01.png" alt=""--></h1>
<p>
<diary_snap_cut>
<span>芝右 外1800m / 天候 : 曇 / 芝 : 良 / 発走 : 14:25</span><br/>
<diary_snap_cut>
<a href="/?pid=race_list&amp;word=%B7%AF%BB%D2%CD%F6%BE%DE&amp;front=1">過去の君子蘭賞</a>
</diary_snap_cut>
</diary_snap_cut></p>
</dd>
</dl>
<p class="smalltxt">2023年3月25日 2回阪神1日目 3歳1勝クラス  牝(特指)(馬齢)</p>
<diary_snap_cut>
<ul class="btn_link_list fc">
<li class="race_btn_MOVIE end">
<a href="/race/movie/202309020109"><img alt="レース映像" class="imgover" src="https://cdn.netkeiba.com/img.db//style/netkeiba.ja/image/race_btn_movie_on.png"/></a>
</li>
</ul>
</diary_snap_cut>
</div>

In [22]:
soup_info.find("h1").text

'君子蘭賞(1勝)'

In [31]:
temp=soup_info.find("p").text.replace(" ","")
temp

'\n\n芝右外1800m\xa0/\xa0天候:曇\xa0/\xa0芝:良\xa0/\xa0発走:14:25\n\n過去の君子蘭賞\n\n'

In [32]:
re.findall(r"\w+",temp)

['芝右外1800m', '天候', '曇', '芝', '良', '発走', '14', '25', '過去の君子蘭賞']

In [33]:
re.findall(r"[\w:]+",temp)

['芝右外1800m', '天候:曇', '芝:良', '発走:14:25', '過去の君子蘭賞']

In [38]:
tmp=soup_info.find_all("p")[1].text
tmp

'2023年3月25日 2回阪神1日目 3歳1勝クラス\xa0\xa0牝(特指)(馬齢)'

In [39]:
re.findall(r"\w+",tmp)

['2023年3月25日', '2回阪神1日目', '3歳1勝クラス', '牝', '特指', '馬齢']

In [57]:
info_dict={}
info_dict["title"]=soup_info.find("h1").text
info_dict["info1"]=re.findall(
    r"[\w:]+",soup_info.find("p").text.replace(" ","")
    )
info_dict["info2"]=re.findall(r"\w+", soup_info.find_all("p")[1].text)
info_dict

{'title': '君子蘭賞(1勝)',
 'info1': ['芝右外1800m', '天候:曇', '芝:良', '発走:14:25', '過去の君子蘭賞'],
 'info2': ['2023年3月25日', '2回阪神1日目', '3歳1勝クラス', '牝', '特指', '馬齢']}

In [58]:
pd.DataFrame().from_dict(info_dict,orient="index").T

,title,info1,info2
0,君子蘭賞(1勝),"[芝右外1800m, 天候:曇, 芝:良, 発走:14:25, 過去の君子蘭賞]","[2023年3月25日, 2回阪神1日目, 3歳1勝クラス, 牝, 特指, 馬齢]"


In [59]:
import create_rawdf

race_info=create_rawdf.create_race_info(html_path_list)

  0%|          | 0/3456 [00:00<?, ?it/s]

In [60]:
race_info

,race_id,title,info1,info2
0,202309020109,君子蘭賞(1勝),"[芝右外1800m, 天候:曇, 芝:良, 発走:14:25, 過去の君子蘭賞]","[2023年3月25日, 2回阪神1日目, 3歳1勝クラス, 牝, 特指, 馬齢]"
1,202303020408,3歳以上1勝クラス,"[芝右1800m, 天候:曇, 芝:良, 発走:14:05]","[2023年7月9日, 2回福島4日目, 3歳以上1勝クラス, 混, 特指, 定量]"
2,202307020402,3歳未勝利,"[ダ左1900m, 天候:晴, ダート:重, 発走:10:20]","[2023年3月19日, 2回中京4日目, 3歳未勝利, 指, 馬齢]"
3,202308010304,3歳未勝利,"[ダ右1800m, 天候:曇, ダート:良, 発走:11:35]","[2023年4月29日, 1回京都3日目, 3歳未勝利, 混, 指, 馬齢]"
4,202304040412,3歳以上1勝クラス,"[芝直線1000m, 天候:晴, 芝:稍重, 発走:16:01]","[2023年10月22日, 4回新潟4日目, 3歳以上1勝クラス, 混, 指, 定量]"
...,...,...,...,...
3451,202305020404,3歳未勝利,"[芝左1400m, 天候:曇, 芝:良, 発走:11:25]","[2023年4月30日, 2回東京4日目, 3歳未勝利, 指, 馬齢]"
3452,202310010810,由布院特別(2勝),"[ダ右1700m, 天候:晴, ダート:良, 発走:14:50, 過去の由布院特別]","[2023年2月5日, 1回小倉8日目, 4歳以上2勝クラス, 混, ハンデ]"
3453,202309010503,3歳未勝利,"[ダ右1400m, 天候:晴, ダート:稍重, 発走:10:55]","[2023年2月25日, 1回阪神5日目, 3歳未勝利, 馬齢]"
3454,202307020105,3歳未勝利,"[芝左2000m, 天候:晴, 芝:良, 発走:12:05]","[2023年3月11日, 2回中京1日目, 3歳未勝利, 指, 馬齢]"


##単勝を予想

In [17]:
import scraping
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
%autoreload

In [3]:
race_id_list=scraping.scrape_race_id_list(["20250111"])

  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
len(race_id_list)

24

In [9]:
url="https://race.netkeiba.com/race/shutuba.html?race_id=202506010401&rf=race_list"

headers={"User-Agent":"Mozilla/5.0"}
request=Request(url, headers=headers)
html = urlopen(request).read()

In [10]:
html

b'<!DOCTYPE html>\n<html>\n<head>\n<meta charset="EUC-JP">\n\n<!-- block=common__meta_tag_each_race (cg) -->\n<meta http-equiv="content-language" content="ja">\n<meta http-equiv="X-UA-Compatible" content="IE=edge"/>\n<meta name="viewport" content="width=1000">\n<meta name="format-detection" content="telephone=no" />\n<meta name="description" content="2025\xc7\xaf1\xb7\xee12\xc6\xfc \xc3\xe6\xbb\xb31R \xa3\xb3\xba\xd0\xcc\xa4\xbe\xa1\xcd\xf8\xa4\xce\xbd\xd0\xc7\xcf\xc9\xbd\xa4\xc7\xa4\xb9\xa1\xa3JRA\xb3\xab\xba\xc5\xa5\xec\xa1\xbc\xa5\xb9\xa4\xce\xbd\xd0\xc7\xcf\xc9\xbd\xa4\xe4\xba\xc7\xbf\xb7\xa5\xaa\xa5\xc3\xa5\xba\xa1\xa2\xa5\xec\xa1\xbc\xa5\xb9\xb7\xeb\xb2\xcc\xc2\xae\xca\xf3\xa1\xa2\xca\xa7\xcc\xe1\xbe\xf0\xca\xf3\xa4\xf2\xa4\xcf\xa4\xb8\xa4\xe1\xa1\xa2\xb6\xa5\xc7\xcf\xcd\xbd\xc1\xdb\xa4\xe4\xa5\xc7\xa1\xbc\xa5\xbf\xca\xac\xc0\xcf\xa4\xca\xa4\xc9\xcd\xbd\xc1\xdb\xa4\xcb\xcc\xf2\xce\xa9\xa4\xc4\xbe\xf0\xca\xf3\xa4\xe2\xcb\xfe\xba\xdc\xa4\xc7\xa4\xb9\xa1\xa3" />\n<meta name="keyword

In [11]:
soup=BeautifulSoup(html,"lxml")
soup

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/>
<!-- block=common__meta_tag_each_race (cg) -->
<meta content="ja" http-equiv="content-language"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=1000" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="2025年1月12日 中山1R ３歳未勝利の出馬表です。JRA開催レースの出馬表や最新オッズ、レース結果速報、払戻情報をはじめ、競馬予想やデータ分析など予想に役立つ情報も満載です。" name="description"/>
<meta content="競馬,keiba,出馬表,オッズ,予想,レース結果,払戻し,結果R報,競馬?想,ネット競馬,netkeiba" name="keywords"/>
<meta content="https://www.netkeiba.com/style/netkeiba.ja/image/netkeiba.png" name="thumbnail"/>
<!-- ogp用 -->
<meta content="netkeiba" property="og:site_name"/>
<meta content="article" property="og:type"/>
<meta content="３歳未勝利 出馬表 | 2025年1月12日 中山1R レース情報(JRA) - netkeiba" property="og:title"/>
<meta content="https://race.netkeiba.com/race/shutuba.html?race_id=202506010401" property="og:url"/>
<meta content="2025年1月12日 中山1R ３歳未勝利の出馬表です。JRA開催レースの出馬表や最新オッズ、レース結果速報、払戻情報をはじめ、競

In [13]:
td_list=soup.find_all("td",class_="HorseInfo")
td_list

[<td class="HorseInfo">
 <div>
 <div>
 <span class="HorseName"><a href="https://db.netkeiba.com/horse/2022101341" target="_blank" title="エコロルキナ">エコロルキナ<img alt="" class="disp_none Favorite" id="myhorse_2022101341" src="https://cdn.netkeiba.com/img.race/common/img/icon/icon_horse.png?2019073001" width="18"/></a></span>
 <img class="disp_none horsebookmark myhorse_2022101341" id="myhorse1_2022101341" onmouseout="this.src='https://cdn.netkeiba.com/img.race/common/img/icon/horse_favorite_icon_pc.png'" onmouseover="this.src='https://cdn.netkeiba.com/img.race/common/img/icon/horse_favorite_icon_pc.png'" src="https://cdn.netkeiba.com/img.race/common/img/icon/horse_favorite_icon_pc.png" width="20"/>
 </div>
 </div>
 </td>,
 <td class="HorseInfo">
 <div>
 <div>
 <span class="HorseName"><a href="https://db.netkeiba.com/horse/2022104122" target="_blank" title="スカイダイバー">スカイダイバー<img alt="" class="disp_none Favorite" id="myhorse_2022104122" src="https://cdn.netkeiba.com/img.race/common/img/icon/icon

In [14]:
td=td_list[0]
td

<td class="HorseInfo">
<div>
<div>
<span class="HorseName"><a href="https://db.netkeiba.com/horse/2022101341" target="_blank" title="エコロルキナ">エコロルキナ<img alt="" class="disp_none Favorite" id="myhorse_2022101341" src="https://cdn.netkeiba.com/img.race/common/img/icon/icon_horse.png?2019073001" width="18"/></a></span>
<img class="disp_none horsebookmark myhorse_2022101341" id="myhorse1_2022101341" onmouseout="this.src='https://cdn.netkeiba.com/img.race/common/img/icon/horse_favorite_icon_pc.png'" onmouseover="this.src='https://cdn.netkeiba.com/img.race/common/img/icon/horse_favorite_icon_pc.png'" src="https://cdn.netkeiba.com/img.race/common/img/icon/horse_favorite_icon_pc.png" width="20"/>
</div>
</div>
</td>

In [15]:
re.findall(r"\d{10}",td.find("a")["href"])[0]

'2022101341'

In [2]:
import create_prediction_population
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
autoreload

In [24]:
race_id_list[0]

NameError: name 'race_id_list' is not defined

In [10]:
create_prediction_population.scrape_horse_id_list(race_id_list[0])

NameError: name 'race_id_list' is not defined

In [25]:
prediction_population=create_prediction_population.create(kaisai_date="20250112")

scraping race_id_list...


  0%|          | 0/1 [00:00<?, ?it/s]

scraping horse_id_list...


  0%|          | 0/24 [00:00<?, ?it/s]

In [12]:
prediction_population

,date,race_id,horse_id
0,2025-01-12,202506010401,2022101341
1,2025-01-12,202506010401,2022104122
2,2025-01-12,202506010401,2022105968
3,2025-01-12,202506010401,2022101484
4,2025-01-12,202506010401,2022103012
...,...,...,...
13,2025-01-12,202507010412,2020104015
14,2025-01-12,202507010412,2021110023
15,2025-01-12,202507010412,2020104739
16,2025-01-12,202507010412,2021105794


In [18]:
html_paths_horse=scraping.scrape_html_horse(
    horse_id_list=prediction_population["horse_id"].unique(),
    skip=False)

  0%|          | 0/388 [00:00<?, ?it/s]

In [19]:
html_paths_horse

[PosixPath('../data/html/horse/2022101341.bin'),
 PosixPath('../data/html/horse/2022104122.bin'),
 PosixPath('../data/html/horse/2022105968.bin'),
 PosixPath('../data/html/horse/2022101484.bin'),
 PosixPath('../data/html/horse/2022103012.bin'),
 PosixPath('../data/html/horse/2022106429.bin'),
 PosixPath('../data/html/horse/2022100239.bin'),
 PosixPath('../data/html/horse/2022101060.bin'),
 PosixPath('../data/html/horse/2022100588.bin'),
 PosixPath('../data/html/horse/2022105295.bin'),
 PosixPath('../data/html/horse/2022101542.bin'),
 PosixPath('../data/html/horse/2022100568.bin'),
 PosixPath('../data/html/horse/2022105913.bin'),
 PosixPath('../data/html/horse/2022103151.bin'),
 PosixPath('../data/html/horse/2022103024.bin'),
 PosixPath('../data/html/horse/2022103332.bin'),
 PosixPath('../data/html/horse/2022100162.bin'),
 PosixPath('../data/html/horse/2022101023.bin'),
 PosixPath('../data/html/horse/2022103839.bin'),
 PosixPath('../data/html/horse/2022104161.bin'),
 PosixPath('../data/

In [20]:
len(html_paths_horse)

388

In [22]:
import create_rawdf
horse_resultws_prediction=create_rawdf.create_horse_results(
    html_path_list=html_paths_horse,
    save_filename="horse_results_prediction.csv"
)

  0%|          | 0/388 [00:00<?, ?it/s]

table not found at 2022104983
table not found at 2022101421
table not found at 2022107016
table not found at 2022102874
table not found at 2022105650
table not found at 2022102280
table not found at 2022101819
table not found at 2022107006
table not found at 2022106164
table not found at 2022107250
table not found at 2022107085
table not found at 2022100345
table not found at 2022101255
table not found at 2022106475
table not found at 2022107013
table not found at 2022100652
table not found at 2022104583
table not found at 2022100521
table not found at 2022101152
table not found at 2022105042
table not found at 2022110094
table not found at 2022101114
table not found at 2022103234
table not found at 2022103495
table not found at 2022104496
table not found at 2022103156
table not found at 2022103483
table not found at 2022110139
table not found at 2022102839
table not found at 2022103052
table not found at 2022101947
